# Splunk App for Data Science and Deep Learning - Notebook for TensorFlow and ONNX model export

## Neural Network for Binary Classification
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk. As an example we use a custom binary neural network classifier built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [3]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import onnx
import tf2onnx
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)
print("ONNX version: " + onnx.__version__)
print("tf2onnx version: " + tf2onnx.__version__)

numpy version: 1.24.2
pandas version: 1.5.3
TensorFlow version: 2.11.0
Keras version: 2.11.0
ONNX version: 1.13.0
tf2onnx version: 1.13.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup diabetes.csv <br>| fit MLTKContainer mode=stage epochs=10 response from * algo=binary_nn_classifier_onnx into app:diabetes_nn_classifier_onnx

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("diabetes_nn_classifier_onnx")
print(str(param))
df


{'options': {'params': {'mode': 'stage', 'epochs': '10', 'algo': 'binary_nn_classifier_onnx'}, 'args': ['response', '*'], 'target_variable': ['response'], 'feature_variables': ['*'], 'model_name': 'diabetes_nn_classifier_onnx', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '10000000', 'max_memory_usage_mb': '16000', 'max_model_size_mb': '3000', 'max_score_time': '6000', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['number_pregnant', 'glucose_concentration', 'blood_pressure', 'skin_thickness', 'serum_insulin', 'BMI', 'diabetes_pedigree', 'age'], 'target_variables': ['response']}


,number_pregnant,glucose_concentration,blood_pressure,skin_thickness,serum_insulin,BMI,diabetes_pedigree,age,response
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## Stage 2 - create and initialize a model

In [7]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    input_shape = int(X.shape[1])
    model_structure = '2-2'
    if 'options' in param:
        if 'params' in param['options']:
            if 'structure' in param['options']['params']:
                model_structure = str(param['options']['params']['structure']).lstrip("\"").rstrip("\"")
    hidden_factors = np.floor(np.array(model_structure.split("-"), dtype="float") * X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.Dense(input_shape, input_dim=input_shape, activation=tf.nn.relu))
    for hidden in hidden_factors:
        model.add(keras.layers.Dense(int(hidden), activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (768, 8)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________
None


2023-02-09 11:34:09.810803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-09 11:34:09.810889: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-09 11:34:09.810920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7df91fd78853): /proc/driver/nvidia/version does not exist
2023-02-09 11:34:09.812477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Stage 3 - fit the model

In [9]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    Y = df[param['target_variables']]
    model_epochs = 100
    model_batch_size = None
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=Y, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = Y)
    return returns

In [10]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/10
24/24 - 1s - loss: 1.9539 - accuracy: 0.5312 - 1s/epoch - 44ms/step
Epoch 2/10
24/24 - 0s - loss: 1.1161 - accuracy: 0.5716 - 102ms/epoch - 4ms/step
Epoch 3/10
24/24 - 0s - loss: 0.8916 - accuracy: 0.5938 - 108ms/epoch - 5ms/step
Epoch 4/10
24/24 - 0s - loss: 0.7706 - accuracy: 0.6263 - 97ms/epoch - 4ms/step
Epoch 5/10
24/24 - 0s - loss: 0.7808 - accuracy: 0.6341 - 84ms/epoch - 4ms/step
Epoch 6/10
24/24 - 0s - loss: 0.8319 - accuracy: 0.5990 - 84ms/epoch - 4ms/step
Epoch 7/10
24/24 - 0s - loss: 0.6807 - accuracy: 0.6523 - 83ms/epoch - 3ms/step
Epoch 8/10
24/24 - 0s - loss: 0.6239 - accuracy: 0.6602 - 88ms/epoch - 4ms/step
Epoch 9/10
24/24 - 0s - loss: 0.6211 - accuracy: 0.6771 - 84ms/epoch - 3ms/step
Epoch 10/10
24/24 - 0s - loss: 0.6124 - accuracy: 0.6719 - 90ms/epoch - 4ms/step
24/24 [==============================] - 0s 1ms/step - loss: 0.5964 - accuracy: 0.6875
[0.5963831543922424, 0.6875]


## Stage 4 - apply the model

In [11]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    y_hat = model.predict(x = X, verbose=1)
    return y_hat

In [12]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

24/24 [==============================] - 0s 1ms/step
[[0.798409  ]
 [0.16197911]
 [0.6890824 ]
 [0.20158887]
 [0.71785015]
 [0.19509788]
 [0.2846882 ]
 [0.71098465]
 [0.50378644]
 [0.23053871]
 [0.06138055]
 [0.65481555]
 [0.60193926]
 [0.9316426 ]
 [0.4388608 ]
 [0.6136509 ]
 [0.3361352 ]
 [0.23190448]
 [0.6698136 ]
 [0.16939822]
 [0.35837492]
 [0.0567136 ]
 [0.4951001 ]
 [0.15004183]
 [0.35118237]
 [0.2893136 ]
 [0.56096756]
 [0.3199977 ]
 [0.08973431]
 [0.09546591]
 [0.47247142]
 [0.507235  ]
 [0.09957866]
 [0.04473288]
 [0.49916166]
 [0.36559373]
 [0.41528502]
 [0.39905393]
 [0.2282671 ]
 [0.29171684]
 [0.7959577 ]
 [0.32584524]
 [0.12097658]
 [0.24756685]
 [0.6480842 ]
 [0.48310357]
 [0.564506  ]
 [0.11920486]
 [0.46136144]
 [0.63036305]
 [0.1073771 ]
 [0.7272074 ]
 [0.5311317 ]
 [0.3175771 ]
 [0.45022857]
 [0.2808242 ]
 [0.6057831 ]
 [0.25502172]
 [0.39506394]
 [0.197618  ]
 [0.50351024]
 [0.5249047 ]
 [0.02385952]
 [0.4591024 ]
 [0.44553587]
 [0.20696352]
 [0.09073886]
 [0.02712

## Stage 5 - save the model

In [13]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 6 - load the model

In [14]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 7 - provide a summary of the model

In [15]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## Export DSDL model to ONNX and use in MLTK for inference
Since MLTK 5.4 you can import ONNX models into MLTK to do inference directly on Splunk. This requires you to create an ONNX model. In this section we do an export of the trained TensorFlow model so it can be imported into MLTK 5.4. This shows the interoperability between DSDL and MLTK via ONNX.

In [16]:
import tf2onnx

In [22]:
# Export our keras model from above as ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, MODEL_DIRECTORY + "dsdl_tf_model.onnx")

2023-02-09 11:55:04.701679: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-02-09 11:55:04.738393: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


Once you downloaded the ONNX model file from Juypter you can upload it into MLTK. Please refer to MLTK documentation here: https://docs.splunk.com/Documentation/MLApp/5.4.0/User/ONNXmodels

You need to specify the ONNX model name and the feature and target variables. The two cells below generate those strings for you so you can copy and paste them into the MLTK ONNX upload dialog.

In [35]:
MLTK_ONNX_target_variable_string = param['target_variables'][0]
print(MLTK_ONNX_target_variable_string)

response


In [36]:
MLTK_ONNX_feature_variable_string = ",".join(param['feature_variables'])
print(MLTK_ONNX_feature_variable_string)

number_pregnant,glucose_concentration,blood_pressure,skin_thickness,serum_insulin,BMI,diabetes_pedigree,age


Once you have your ONNX model successfully imported into MLTK you can run inference with the following SPL and you should see a response field getting added to your results.

| inputlookup diabetes.csv<br/>
| apply onnx:dsdl_tf_model

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code